In [1]:
import mujoco as mu
import os
import numpy as np
import mujoco.viewer as vi
import mediapy as media
xml2 = """
<mujoco>
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <default>
    <joint type="hinge" axis="0 -1 0"/>
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <light pos="0 -.4 1"/>
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    <body name="0" pos="0 0 .2">
      <joint name="root"/>
      <geom fromto="-.2 0 0 .2 0 0" rgba="1 1 0 1"/>
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>
      <body name="1" pos="-.2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="1 0 0 1"/>
      </body>
      <body name="2" pos=".2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 1 0 1"/>
      </body>
      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""

xml3 = """
<mujoco>
  <visual>
    <global elevation="-10"/>
  </visual>

  <compiler angle="radian"/>

  <!-- Note: actuator groups 1, 2, are disabled by default -->
  <option actuatorgroupdisable="1 2" cone="elliptic"/>

  <asset>
    <texture type="skybox" builtin="gradient" rgb1=".3 .5 .7" rgb2="0 0 0" width="32" height="512"/>
    <texture name="body" type="cube" builtin="flat" mark="cross" width="128" height="128" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" markrgb="1 1 1" random="0.01"/>
    <material name="body" texture="body" texuniform="true" rgba="0.8 0.6 .4 1"/>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="6 2" texuniform="false" reflectance=".2"/>
  </asset>

  <default>
    <general ctrlrange="-1 1"/>
    <geom friction=".4"/>
    <default class="decor">
      <geom type="cylinder" size=".03" fromto="0 -.03 0 0 .03 0" rgba="1 1 1 1"/>
    </default>
    <default class="arm">
      <joint damping="5" axis="0 -1 0"/>
      <geom type="capsule" size=".02" material="body"/>
    </default>
    <default class="object">
      <geom type="cylinder" size="0.1" rgba="0 .9 0 1" fromto="0 -.03 0 0 .03 0"
            solref="-10000 -10" priority="1"/>
      <joint group="3"/>
    </default>

    <!-- Note: motors, position, intvelocity actuators are in groups 0, 1, 2, respectively -->
    <default class="motor">
      <motor group="0" gear="30" ctrlrange="-1 1"/>
    </default>
    <default class="position">
      <position group="1" kp="60"/>
    </default>
    <default class="intvelocity">
      <intvelocity group="2" kp="60" ctrlrange="-.5 .5"/>
    </default>
  </default>

  <worldbody>
    <geom name="floor" type="plane" size=".6 .2 0.01" material="grid"/>
    <geom name="wall0" type="plane" size=".4 .2 0.01" material="grid" pos="-.6 0 .4" zaxis="1 0 0"/>
    <geom name="wall1" type="plane" size=".4 .2 0.01" material="grid" pos=".6 0 .4" zaxis="-1 0 0"/>
    <light pos="-.6 0 3" mode="targetbody" target="hand"/>
    <light pos=".6 0 3" mode="targetbody" target="hand"/>
    <body name="arm" pos="-.6 0 .6" childclass="arm">
      <joint name="arm" range="-1.5 1.5"/>
      <geom class="decor"/>
      <geom name="arm" fromto="0 0 0 .7 0 0"/>
      <body pos=".7 0 0">
        <joint name="hand" range="-4.65 1.5"/>
        <geom class="decor"/>
        <geom fromto="0 0 0 0 0 0.4"/>
        <body name="hand" pos="0 0 0.4">
          <geom name="hand" type="sphere" size=".06" mass="0"/>
        </body>
      </body>
    </body>

    <body childclass="object" pos="-.3 0 .2">
      <joint axis="1 0 0" type="slide"/>
      <joint axis="0 0 1" type="slide"/>
      <joint axis="0 1 0"/>
      <geom/>
    </body>
    <body childclass="object" pos="-.2 0 .4">
      <joint axis="1 0 0" type="slide"/>
      <joint axis="0 0 1" type="slide"/>
      <joint axis="0 1 0"/>
      <geom/>
    </body>
  </worldbody>

  <contact>
    <exclude body1="world" body2="arm"/>
    <pair geom1="arm" geom2="hand"/>
  </contact>

  <actuator>
    <motor class="motor" name="arm torque" joint="arm"/>
    <motor class="motor" name="hand torque" joint="hand"/>
    <position class="position" name="arm position" joint="arm" ctrlrange="-1.5 1.5"/>
    <position class="position" name="hand position" joint="hand" ctrlrange="-1.5 4.65"/>
    <intvelocity class="intvelocity" name="arm velocity" joint="arm" actrange="-1.5 1.5"/>
    <intvelocity class="intvelocity" name="hand velocity" joint="hand" actrange="-1.5 4.65"/>
  </actuator>
</mujoco>
"""

xml4 = """
<mujoco>
    <asset>
        <!-- Define ground material -->
        <material name="ground" rgba="0.8 0.9 0.8 1"/>
    </asset>
    <option gravity ="0 0 -9.82"/>
    
    <worldbody>
        <!-- Define the ground -->
        <geom name="ground" type="plane" size="1 1 0.1" material="ground"/>

        <!-- Define the box -->
        <body name="box" pos="0 0 0.5">
            <joint type="free"/>
            <geom name="box_geom" type="box" size="0.1 0.1 0.1" rgba="0.5 0.5 0.5 1"/>

            <!-- Define the rod/cable -->
            <body name="rod" pos="0 0 0.5">
                <joint name = "rodj" type="slide" axis="0 0 -1"/>
                <geom name="rod_geom" type="capsule" size="0.01 0.01" fromto="0 0 0.5 0 0 1.5" rgba="0.2 0.2 0.2 1"/>
            </body>
        </body>
    </worldbody>

    <!-- Define the actuators -->
    <actuator>
        <velocity name="vel" joint="rodj"/>
    </actuator>
</mujoco>
"""

xml5 = """
<mujoco>
    <asset>
        <!-- Define ground material -->
        <material name="ground" rgba="0.8 0.9 0.8 1"/>
    </asset>
    <option gravity ="0 0 -1"/>
    
    <worldbody>
        <!-- Define the ground -->
        <geom name="ground" type="plane" size="1 1 0.1" material="ground"/>
        <site name="anchor" pos="0 0 1" size=".01"/>
        <site name="anchor2" pos="0.2 0 1" size=".01"/>
        <site name="anchor3-5" pos="0.1 -0.2 1" size=".01"/>

        <body name="fuk" pos="0.1 -0.4 1">
            <joint name="fuk2" type="slide" axis="0 0 1"/>
            <geom type="box" size=".02 .02 .02"/>
            <site name="anchor3" pos="0.1 -0.4 1" size=".01"/>
        </body>


        <!-- Define the box -->
        <body name="box" pos="0 0 0.5">
            <joint axis="0 0 1" type="free"/>
            <geom name="box_geom" type="box" size="0.1 0.1 0.1" rgba="0.5 0.5 0.5 1"/>
            <site name="hook" pos="-.1 -.1 -.1" size=".01"/>
            <site name="hook2" pos="-.1 .1 -.1" size=".01"/>
            <site name="hook3" pos="-.1 -.03 .1" size=".01"/>
        </body>
    </worldbody>

    <tendon>
        <spatial name="wire" limited="true" range="0 0.35" width="0.003">
            <site site="anchor"/>
            <site site="hook"/>
        </spatial>
    </tendon>

    <tendon>
        <spatial name="wire2" limited="true" range="0 0.35" width="0.003">
            <site site="anchor2"/>
            <site site="hook2"/>
        </spatial>
    </tendon>

    <tendon>
        <spatial name="wire3" limited="true" range="0 0.35" width="0.003">
            <site site="anchor3"/>
            <site site="hook3"/>
        </spatial>
    </tendon>

    <actuator>
        <position name="speed" joint="fuk2"/>
    </actuator>
</mujoco>
"""

model = mu.MjModel.from_xml_string(xml5)
data = mu.MjData(model)
mu.mj_kinematics(model,data)

duration = 10 # (seconds)
framerate = 60  # (Hz)

# enable joint visualization option:
scene_option = mu.MjvOption()
scene_option.flags[mu.mjtVisFlag.mjVIS_JOINT] = True
rend = mu.Renderer(model)

# Simulate and display video.
frames = []
mu.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mu.mj_step(model, data)
  if len(frames) < data.time * framerate:
    rend.update_scene(data)
    pixels = rend.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

vi.launch(model)

This browser does not support the video tag.
